## Actividad 2
En la presente actividad vamos a entrenar un modelo basado en redes neuronales utilizando tensorflow y keras



### Librerias a importar

In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, InputLayer

import matplotlib.pyplot as plt

## Carga del dataset



In [ ]:
path = 'https://raw.githubusercontent.com/eyberthrojas/deeplearning-actividad-2/main/poker_dataset.csv'
df = pd.read_csv(path)
df.sample(5, random_state=10)

Vamos a construir un modelo que infiera la figura en una mano de poker, es decir, en una mano de poker tenemos 5 cartas, el modelo debe decir a que figura corresponde, ej: si en una mano tenemos 3 de picas, 3 de corazones, 3 de treboles, 10 de picas y 8 de diamantes, la figura resultante es un trio 

En el dataset cada carta tiene SUIT_n y RANK_n se refieren al color y al valor de la carta n, la columna Figura corresponde a la figura que hay en el juego de cartas, esta columna "Figura" es el Label o etiqueta para entrenar el modelo supervisado. 

In [ ]:
# Colores
list(df['SUIT_1'].unique())

In [ ]:
# Valores
list(df['RANK_1'].unique())

In [ ]:
# Figuras
list(df['Figura'].unique())

In [ ]:
# Distribución de datos respecto a la figura
df['Figura'].value_counts()

In [9]:
dataset = df.copy(deep=True)

### Preprocesamiento, 
Antes de empezar a entrenar, se requiere codificar las variables y el Label de manera numérica, en este caso es sencillo asignarle un codigo al color, al valor y a la Figura si necesidad de utilizar algún encoder:

In [10]:
figura = {
    'no_hay_figura': 0,
    'pareja': 1,
    'doble_pareja': 2,
    'trio': 3,
    'escalera': 4,
    'color': 5,
    'full': 6,
    'poker': 7,
    'escalera_color': 8,
    'escalera_real': 9    
}
color = {
    'corazones': 0,
    'diamantes': 1,
    'picas': 2,
    'treboles': 3,
}
numero = {
    '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, '10': 10, 'J': 11, 'Q': 12, 'K': 13
}

### Codificación variables de entrada (SUIT_n, RANK_n)

In [11]:
for card in range(1,6):
  dataset['SUIT_'+str(card)] = dataset['SUIT_'+str(card)].apply(lambda x:color[x])
  dataset['RANK_'+str(card)] = dataset['RANK_'+str(card)].apply(lambda x:numero[x])

### Codificación Label (Figura)

In [12]:
dataset['Figura'] = dataset['Figura'].apply(lambda x:figura[x])

In [ ]:
dataset.sample(5, random_state=10)

## División de datos para entrenamiento y prueba

In [14]:
df_train, df_test, _, _ = train_test_split(dataset, dataset['Figura'], test_size=0.2, stratify=dataset['Figura'], random_state=0)

In [15]:
features = [
    'SUIT_1', 'RANK_1', 'SUIT_2', 'RANK_2', 'SUIT_3', 'RANK_3', 
    'SUIT_4', 'RANK_4', 'SUIT_5', 'RANK_5'
]
X_train = df_train[features].to_numpy()
y_train = df_train['Figura'].to_numpy()
X_test = df_test[features].to_numpy()
y_test = df_test['Figura'].to_numpy()

In [ ]:
print(f'Tamaño X_train = {X_train.shape}')
print(f'Tamaño X_test = {X_test.shape}')

Ya estn los datos, ahora a construir el modelo que vamos a entrenar

## Definición de hyperparámetros


In [17]:
# Número de caracteristicas
n_features = len(features)
# Número de clases
n_classes = len(np.unique(y_train))
# Numero de neuronas por capa
neurons = [128, 256, n_classes]
# unciones de activación para la respectiva capa
activations = ['tanh', 'relu', 'softmax']
# learning rate
learning_rate=0.001
# Número de épocas
epochs=200

## Construcción del modelo

In [18]:
model = Sequential()
model.add(InputLayer((n_features,)))
for neuron, activation in zip(neurons, activations):
  model.add(Dense(neuron, activation=activation))

In [ ]:
model.summary()

## Función de costo
Cómo es un clasiicador multiclase, utilizamos sparce categorical cross entropy 

In [20]:
# Función de costo 
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
# Optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
model.compile(optimizer=optimizer, loss=loss, metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])


### Entrenando el modelo

In [21]:
history = model.fit(x=X_train, y=y_train, batch_size=32, epochs=epochs, validation_data=(X_test, y_test), verbose=False)

In [ ]:
history.history.keys()

In [23]:
epochs_v = range(1,epochs+1)

## Grafica de perdida respecto a las epocas,

In [ ]:
plt.plot(epochs_v, history.history['loss'], epochs_v, history.history['val_loss'])
plt.legend(['train_loss', 'test_loss'])

In [ ]:
plt.plot(epochs_v, history.history['sparse_categorical_accuracy'], epochs_v, history.history['val_sparse_categorical_accuracy'])
plt.legend(['train_accuracy', 'test_accuracy'])

Reporte de cada clase

In [ ]:
probs = model.predict(X_test)

In [26]:
preds = np.argmax(probs, axis=1)

El reporte de clasificacion nos entrega las metricas totales de cada clase en terminos de precision, recall, f1-score y accuracy. La columna support se refiere a la cantidad de datos verdaderos de cada clase

In [ ]:
print(classification_report(y_test, preds, zero_division=0))